# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [ ]:
# constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [ ]:
# Configura el entorno

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")

openai = OpenAI()

In [ ]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

system_prompt = "Necesito que actues como un programador experto en Python. \
Responde lo más detallado posible, dando ejemplos. \
Responde en formato markdown.\n"

user_prompt = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Haz que gpt-4o-mini responda, con streaming

def python_expert():
    # Llamada al modelo con streaming habilitado
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True  # Habilitar streaming
    )

    # Inicializar variables para mostrar el contenido progresivamente
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    # Procesar cada fragmento de la respuesta
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response


# Llamada a la función
final_response = python_expert()
print("Respuesta final completa:")
print(final_response)


In [ ]:
# Haz que Llama 3.2 responda

messages = [
    {
        "role": "user",
        "content": (
            "Necesito que actúes como un programador experto en Python. "
            "Responde lo más detallado posible, dando ejemplos. "
            "Responde en formato markdown. "
            "Explica qué hace este código y por qué: "
            "yield from {book.get('author') for book in books if book.get('author')}"
        )
    }
]

payload = {
    "model": MODEL_LLAMA,
    "messages": messages,
    "stream": False
}

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)

# Procesar la respuesta JSON
response_json = response.json()

# Extraer el contenido de la respuesta
message_content = response_json.get('message', {}).get('content', 'No content received')

# Mostrar en formato Markdown
display(Markdown(message_content))